# Data sanity check process

In this notebook, we will perform a data sanity check: for that purpose, we wil extract a dataset and the content of a `data_format_ref` file and the dataset from which the `data_format_ref` has been created (we will assume it is is dataset we want to check 

we will run a data sanity check on this dataset based on the content of`data_format_ref`

In [1]:
import pandas as pd

import fedbiomed.common.data_tool.utils as utils
import fedbiomed.common.data_tool.multi_view_dataframe as multiview

from fedbiomed.common.data_tool.warning_logger import WarningReportLogger
from fedbiomed.common.data_tool.pre_processing_checker import PreProcessingChecker

## Loading a simple CSV dataset (single view dataset) and its `data_format_ref` file

In [2]:
# loading simple dataset

single_dataset_to_check = utils.load_tabular_datasets(r'data/pseudo_adni/pseudo_adni_mod_2.csv')

# loading `data_format_ref`

single_format_file_ref = utils.load_format_file_ref(r'single_format_file_ref_test2')




file found


In [3]:
single_format_file_ref

{'global_thresholds': {'min_nb_samples': 100, 'min_nb_missing_samples': 50},
 'pseudo_adni_mod_2.csv': {'CDRSB.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'CHARACTER',
   'values': ["<class 'str'>"],
   'is_missing_values': True,
   'data_imputation_method': 'MODE_IMPUTATION',
   'data_imputation_parameters': None,
   'data_imputation_variables': None},
  'ADAS11.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': ['int64'],
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None},
  'MMSE.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': ['float64'],
   'is_missing_values': True,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None},
  'RAVLT.immediate.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': ['float64'],
   'is_missing_values': True,
   'dat

In [4]:
# Simple routine for data sanity checking (here with a simple tabular dataset)

# select only features in dataset that will be checked
pre_parsed_dataset_to_check = multiview.select_data_from_format_file_ref(single_dataset_to_check,
                                                                         single_format_file_ref)

primary_key = multiview.search_primary_key(single_format_file_ref)

if primary_key is None:
    print('not a multi view file')
    
df_joined =  multiview.join_multi_view_dataset(pre_parsed_dataset_to_check, primary_key, False)
warning_report = WarningReportLogger(disclosure=3)

checker = PreProcessingChecker(single_format_file_ref, df_joined,
                               'dataFormat_file' ,
                               warning_report)

#checker.update_views_features_name(new_views_name)
checker.check_all()

not a multi view file
read data type CATEGORICAL
read data type CATEGORICAL
read data type QUANTITATIVE
read data type QUANTITATIVE
read data type QUANTITATIVE


We can get back the repport log: it is a dictionary where for each fields, is specified the test run.

it also indicates the number of warning and exception found when running data check

In [5]:
report = checker.get_warning_logger()
report

number of warnings: 0
Number of error: 0


{'N_SAMPLES_BELOW_THRESHOLD': [{'view': 'pseudo_adni_mod_2.csv',
   'success': True,
   'feature': 'ALL',
   'msg': 'Test passed'}],
 'INCORRECT_FORMAT_FILE': [{'view': 'pseudo_adni_mod_2.csv',
   'success': True,
   'feature': 'CDRSB.bl',
   'msg': 'Test passed'},
  {'view': 'pseudo_adni_mod_2.csv',
   'success': True,
   'feature': 'ADAS11.bl',
   'msg': 'Test passed'},
  {'view': 'pseudo_adni_mod_2.csv',
   'success': True,
   'feature': 'MMSE.bl',
   'msg': 'Test passed'},
  {'view': 'pseudo_adni_mod_2.csv',
   'success': True,
   'feature': 'RAVLT.immediate.bl',
   'msg': 'Test passed'},
  {'view': 'pseudo_adni_mod_2.csv',
   'success': True,
   'feature': 'Ventricles.bl',
   'msg': 'Test passed'}],
 'DATA_TYPE_MISMATCH': [{'view': 'pseudo_adni_mod_2.csv',
   'success': True,
   'feature': 'CDRSB.bl',
   'msg': 'Test passed'},
  {'view': 'pseudo_adni_mod_2.csv',
   'success': True,
   'feature': 'ADAS11.bl',
   'msg': 'Test passed'},
  {'view': 'pseudo_adni_mod_2.csv',
   'success

In [25]:
# raise exception (if found any, otherwise return None)

checker.get_raised_exception()

## Explanations of the tests run for data sanity checks

- 'N_SAMPLES_BELOW_THRESHOLD': check if number of samples in a view are below threshold set in data fromat file
- 'INCORRECT_FORMAT_FILE': check if for a given feature, each feilds have been correctly set (check if data format file has been modified/corrupted)
- 'DATA_TYPE_MISMATCH': check if there is any inconsistance between data format (eg QUANTITATIVE, CATEGORICAL) and subtype (if any (eg DISCRETE, BOOLEAN, ...)
- 'INCORRECT_DATA_TYPE': check if there is a mismatch beween specified data type and actual data type in the dataset for agiven feature
- 'MISSING_DATA_ALLOWED': check if missing data are allowed or not. Should raise an exception when using `get_raised_exception` if test fails
- 'N_MISSING_DATA_ABOVE_THRESHOLD': check for each feature if number of missing data is below threshold. Should raise an exception when using `get_raised_exception` if test fails
- 'OUTLIER_DETECTION_LOWER_BOUND': check for outliers (samples below threshold)
- 'OUTLIER_DETECTION_UPPER_BOUND': check for outliers (samples above threshold)
- 'INCORRECT_VALUES_CATEGORICAL_DATA': check if values in categorical variables 
- INCORRECT_DATETIME_DATA: check if datetime in variable is parseable
- KEY_UNIQUENESS_VIOLATED: check if KEY variable samples are unique.


## Loading a multi view dataset (folder of csv dataset), and its `data_format_ref` file

In [27]:
# loading dataset

multi_dataset_to_check = utils.load_tabular_datasets(r'data/csv_folder')

directory found


In [28]:
# loading `data_fromat_ref`

multi_format_file_ref = utils.load_format_file_ref('int_multi_view_format_test')

In [29]:
multi_format_file_ref

{'global_thresholds': {'min_nb_samples': 10000, 'min_nb_missing_samples': 10},
 'file1': {'a': {'data_format': 'UNKNOWN',
   'data_type': 'UNKNOWN',
   'values': None,
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None},
  'e': {'data_format': 'DATETIME',
   'data_type': 'DATETIME',
   'values': ["<class 'numpy.datetime64'>"],
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None},
  'i': {'data_format': 'UNKNOWN',
   'data_type': 'UNKNOWN',
   'values': None,
   'is_missing_values': True,
   'data_imputation_method': 'LINEAR_INTERPOLATION_IMPUTATION',
   'data_imputation_parameters': None,
   'data_imputation_variables': {'file1': ['a',
     'e',
     'i',
     'o',
     '0',
     '1',
     '2',
     '3',
     'time',
     'pressure',
     'sp02',
     'a.1',
     'e.1',
     'i.1',
     'o.1',
     'gender',

we wil extract only features indicated in format_file_ref

In [30]:
# select only features in dataset that will be checked
pre_parsed_dataset_to_check = multiview.select_data_from_format_file_ref(multi_dataset_to_check,
                                                                         multi_format_file_ref)

the dataset loaded is a dataset composed from multiple CSV files. To convert those dataset into a single one, we will do a join operation based on a primary key

let's  find this primary key: It is the only column that contains the `KEY`Data type

In [31]:
primary_key = multiview.search_primary_key(multi_format_file_ref)

found primary key pkey
found primary key pkey
found primary key pkey


Performing a join operation

In [32]:
# extract views names from format_file_ref
views_names = utils.get_view_names(multi_format_file_ref)

# rename columns names before join operation

pre_parsed_dataset_to_check, new_views_name = multiview.rename_variables_before_joining(pre_parsed_dataset_to_check, views_names,
                                                             primary_key)

In [33]:
# perform a join operation

# jointure operation (takesplace only if primary key has been specfied in foramt_file)
df_joined = multiview.join_multi_view_dataset(pre_parsed_dataset_to_check, primary_key, False)

In [34]:
df_joined

,discrete,city,pkey,a,e,i,o,0,file1.time,pressure,sp02,a.1,gender,blood type,1,file2.time,pH
0,64.0,Lille,qpqorfhylu gmfjy bdj,67,16,54,25,True,2018-01-03 04:00:00,0.986670,0.676041,6,WOMAN,A,False,2018-01-02 06:00:00,NaN
1,26.0,Lille,kkmjozalfyirgsire ui,42,96,69,61,True,2018-01-02 04:00:00,0.996889,0.864713,32,MAN,AB,True,2018-01-01 00:00:00,0.023107
2,61.0,Paris,ezfasuuycdda foisjte,46,8,89,21,False,2018-01-01 09:00:00,0.777026,0.253232,64,MAN,A,False,2018-01-02 10:00:00,0.587685
3,29.0,Paris,faxiqkt xggzmwzoidbg,29,6,77,14,False,2018-01-04 20:00:00,0.877527,0.450811,73,MAN,AB,True,2018-01-03 12:00:00,0.894073
4,99.0,Lille,znwhlj rwzdutnagwasy,96,79,19,33,False,2018-01-04 09:00:00,0.447389,0.639577,85,WOMAN,O,True,2018-01-01 10:00:00,0.026831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9.0,Paris,zeqhcikzdodus jn qjf,81,62,52,68,True,2018-01-02 13:00:00,0.953184,0.439232,34,MAN,AB,False,2018-01-02 05:00:00,0.788560
96,98.0,Marseille,iicthcvfmkajbvr gzir,16,49,30,7,False,2018-01-02 21:00:00,0.442283,0.600981,82,MAN,NaN,True,2018-01-05 02:00:00,0.402979
97,21.0,Lille,ztjakcsk bhjoksdz lm,90,14,36,24,False,2018-01-02 06:00:00,0.988543,0.377624,35,MAN,B,False,2018-01-01 12:00:00,NaN
98,42.0,Marseille,sabunaa opt vpulnxj,91,10,69,58,True,2018-01-02 01:00:00,0.059791,0.490360,42,MAN,B,True,2018-01-02 09:00:00,0.651801


In [35]:
# convert joined dataset into multi view dataset (not needed, just to better display dataset)
new_feature_name = { v: list(pre_parsed_dataset_to_check[v].columns) for v in views_names}
new_feature_name


multiview.create_multi_view_dataframe_from_dataframe(df_joined,new_feature_name, primary_key=primary_key)

length ['a', 'e', 'i', 'o', '0', 'file1.time', 'pressure', 'sp02', 'a.1', 'gender', 'blood type', 'discrete', 'city', '1', 'file2.time', 'pH'] ['file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'contatct', 'contatct', 'file2', 'file2', 'file2']


views        file1                                                    \
feature_name     a   e   i   o      0           file1.time  pressure   
0               67  16  54  25   True  2018-01-03 04:00:00   0.98667   
1               42  96  69  61   True  2018-01-02 04:00:00  0.996889   
2               46   8  89  21  False  2018-01-01 09:00:00  0.777026   
3               29   6  77  14  False  2018-01-04 20:00:00  0.877527   
4               96  79  19  33  False  2018-01-04 09:00:00  0.447389   
..             ...  ..  ..  ..    ...                  ...       ...   
95              81  62  52  68   True  2018-01-02 13:00:00  0.953184   
96              16  49  30   7  False  2018-01-02 21:00:00  0.442283   
97              90  14  36  24  False  2018-01-02 06:00:00  0.988543   
98              91  10  69  58   True  2018-01-02 01:00:00  0.059791   
99              22  89  36  15  False  2018-01-03 22:00:00  0.939352   

views                                        contatct             file2  \
feature_name      sp02 a.1 gender blood type discrete       city      1   
0             0.676041   6  WOMAN          A     64.0      Lille  False   
1             0.864713  32    MAN         AB     26.0      Lille   True   
2             0.253232  64    MAN          A     61.0      Paris  False   
3             0.450811  73    MAN         AB     29.0      Paris   True   
4             0.639577  85  WOMAN          O     99.0      Lille   True   
..                 ...  ..    ...        ...      ...        ...    ...   
95            0.439232  34    MAN         AB      9.0      Paris  False   
96            0.600981  82    MAN        NaN     98.0  Marseille   True   
97            0.377624  35    MAN          B     21.0      Lille  False   
98             0.49036  42    MAN          B     42.0  Marseille   True   
99            0.143049  75    MAN          B      3.0      Paris  False   

views                                                 primary_key  
feature_name           file2.time        pH                  pkey  
0             2018-01-02 06:00:00       NaN  qpqorfhylu gmfjy bdj  
1             2018-01-01 00:00:00  0.023107  kkmjozalfyirgsire ui  
2             2018-01-02 10:00:00  0.587685  ezfasuuycdda foisjte  
3             2018-01-03 12:00:00  0.894073  faxiqkt xggzmwzoidbg  
4             2018-01-01 10:00:00  0.026831  znwhlj rwzdutnagwasy  
..                            ...       ...                   ...  
95            2018-01-02 05:00:00   0.78856  zeqhcikzdodus jn qjf  
96            2018-01-05 02:00:00  0.402979  iicthcvfmkajbvr gzir  
97            2018-01-01 12:00:00       NaN  ztjakcsk bhjoksdz lm  
98            2018-01-02 09:00:00  0.651801   sabunaa opt vpulnxj  
99            2018-01-03 02:00:00  0.751969  qmbexyexvgromrm admu  

[100 rows x 17 columns]

## Perform a data sanity check

In [36]:
multi_format_file_ref

{'global_thresholds': {'min_nb_samples': 10000, 'min_nb_missing_samples': 10},
 'file1': {'a': {'data_format': 'UNKNOWN',
   'data_type': 'UNKNOWN',
   'values': None,
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None},
  'e': {'data_format': 'DATETIME',
   'data_type': 'DATETIME',
   'values': ["<class 'numpy.datetime64'>"],
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None},
  'i': {'data_format': 'UNKNOWN',
   'data_type': 'UNKNOWN',
   'values': None,
   'is_missing_values': True,
   'data_imputation_method': 'LINEAR_INTERPOLATION_IMPUTATION',
   'data_imputation_parameters': None,
   'data_imputation_variables': {'file1': ['a',
     'e',
     'i',
     'o',
     '0',
     '1',
     '2',
     '3',
     'time',
     'pressure',
     'sp02',
     'a.1',
     'e.1',
     'i.1',
     'o.1',
     'gender',

In [37]:
warning_report = WarningReportLogger(disclosure=3)

checker = PreProcessingChecker(multi_format_file_ref,
                               df_joined,
                               'multiview_dataFormat_file' ,
                               warning_report)

checker.update_views_features_name(new_views_name)
checker.check_all()



features names updated
MinimumSamplesViolatedException: Number of samples contained in dataset file1 is below threshold (expected at least 10000 samples, found 100 samples)
read data type UNKNOWN
read data type DATETIME
read data type UNKNOWN
read data type QUANTITATIVE
read data type CATEGORICAL
read data type DATETIME
read data type QUANTITATIVE
read data type QUANTITATIVE
read data type QUANTITATIVE
read data type CATEGORICAL
MissingDataException: Variable blood type must not have missing data, but some were found
read data type CATEGORICAL
read data type KEY
MinimumSamplesViolatedException: Number of samples contained in dataset contatct is below threshold (expected at least 10000 samples, found 100 samples)
read data type QUANTITATIVE
read data type CATEGORICAL
read data type KEY
MinimumSamplesViolatedException: Number of samples contained in dataset file2 is below threshold (expected at least 10000 samples, found 100 samples)
read data type CATEGORICAL
read data type DATETIME
Mis

## get report

In [38]:
report = checker.get_warning_logger()

number of warnings: 5
Number of error: 5


In [16]:
report

{'N_SAMPLES_BELOW_THRESHOLD': [{'view': 'file1',
   'success': False,
   'feature': 'ALL',
   'msg': 'MinimumSamplesViolatedException: Number of samples contained in dataset file1 is below threshold (expected at least 10000 samples, found 100 samples)'},
  {'view': 'contatct',
   'success': False,
   'feature': 'ALL',
   'msg': 'MinimumSamplesViolatedException: Number of samples contained in dataset contatct is below threshold (expected at least 10000 samples, found 100 samples)'},
  {'view': 'file2',
   'success': False,
   'feature': 'ALL',
   'msg': 'MinimumSamplesViolatedException: Number of samples contained in dataset file2 is below threshold (expected at least 10000 samples, found 100 samples)'}],
 'INCORRECT_FORMAT_FILE': [{'view': '',
   'success': True,
   'feature': 'a',
   'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'e', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'i', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'featur

# raise collected error

This will raises all collected error found in the warning logger

In [40]:
checker.get_raised_exception()

DataSanityCheckException: MinimumSamplesViolatedException: Number of samples contained in dataset file1 is below threshold (expected at least 10000 samples, found 100 samples)
MissingDataException: Variable blood type must not have missing data, but some were found
MinimumSamplesViolatedException: Number of samples contained in dataset contatct is below threshold (expected at least 10000 samples, found 100 samples)
MinimumSamplesViolatedException: Number of samples contained in dataset file2 is below threshold (expected at least 10000 samples, found 100 samples)
MissingDataException: Variable pH must not have missing data, but some were found